In [3]:
import pandas as pd
import pandas_profiling

In [4]:
df = pd.read_csv('Using-machine-learning-to-detect-malicious-URLs/data/data.csv')

In [5]:
df.profile_report()

In [9]:
from tqdm import tqdm_notebook as tq
with open('data.txt', 'w') as f:
    for idx, row in tq(df.iterrows(), total=len(df)):
        threat = -1 if row['label']=='bad' else 1
        f.write(str(threat)+'\t'+row['url']+'\n')
        